# SD212: Graph mining
## Lab 7: Graph embedding

In this lab, you will learn to embed the nodes of a graph in a vector space of low dimension.

## Import

In [ ]:
from IPython.display import SVG

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt

In [ ]:
# if you get any error, please update scikit-network!
from sknetwork.data import load_netset, karate_club
from sknetwork.clustering import KMeans
from sknetwork.embedding import Spectral, BiSpectral
from sknetwork.linalg import HalkoEig
from sknetwork.ranking import PageRank, BiPageRank, top_k
from sknetwork.visualization import svg_graph, svg_digraph

## Data

We will work on the following graphs (see the [NetSets](https://graphs.telecom-paristech.fr/Home_page.html#netsets-section) collection for details):
* Openflights (graph)
* WikiVitals (digraph)
* Cinema (bigraph)

In [ ]:
openflights = load_netset('openflights')
wikivitals = load_netset('wikivitals')
cinema = load_netset('cinema')

## 1. Graphs

## Karate Club


We first consider the spectral embedding of the [karate club graph](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

In [ ]:
graph = karate_club(True)

In [ ]:
adjacency = graph.adjacency
position = graph.position
labels_true = graph.labels

In [ ]:
image = svg_graph(adjacency, position, labels=labels_true)
SVG(image)

## To do

* Display the spectrum of the normalized Laplacian (e.g., first 20 eigenvalues). 
* What does the spectrum suggest?
* Display the graph with the embedding in dimension 2.
* Check that the embedding weighted by the degrees is centered.
* How many nodes are misclassified using the sign of the first eigenvector? What are these nodes?

In [ ]:
spectral = Spectral(20, barycenter=False, normalized=False, regularization=None)

In [ ]:
spectral.fit(adjacency)

In [ ]:
eigenvalues = spectral.eigenvalues_

In [ ]:
eigenvalues

Display the spectrum of the normalized Laplacian (e.g., first 20 eigenvalues).

In [ ]:
spectral = Spectral(20, normalized_laplacian=True, barycenter=False, normalized=False, regularization=None)
spectral.fit(adjacency)
eigenvalues_n = spectral.eigenvalues_

In [ ]:
eigenvalues_n

In [ ]:
full_eigenvalues = np.hstack(([0],eigenvalues_n))

In [ ]:
plt.plot(full_eigenvalues, 'o')
plt.grid()

What does the spectrum suggest?

- This eigenvalues from Karate Club suggest that there exists 4 clusters,
the first 4 are not zero because the four clusters are not completely isolated,
but we have a gap from the four eigenvalue.

Display the graph with the embedding in dimension 2 of Karate Club.

In [ ]:
spectral = Spectral(2, normalized_laplacian=True, barycenter=False, normalized=False, regularization=None)
embedding = spectral.fit_transform(adjacency)
plt.plot(embedding[:,0],embedding[:,1], 'o')
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

Check that the embedding weighted by the degrees is centered.

In [ ]:
degree = np.diag(adjacency.toarray().sum(axis=1))
mean = np.dot(degree,embedding)
print ( np.abs(np.mean(mean[:,0])) )
print ( np.abs(np.mean(mean[:,1])) )

In [ ]:
#embedding

In [ ]:
degree

In [ ]:
#mean

In [ ]:
#np.dot(degree,embedding)

- As values are:
-- 3.2653618371328135e-17
-- 2.938825653419532e-16
they are centered.

How many nodes are misclassified using the sign of the first eigenvector? What are these nodes?

In [ ]:
labels_true
first_label = [embedding[:,0] > 0]

In [ ]:
misclass = labels_true - first_label

print(f"There are {np.size(np.nonzero(misclass[0]))} nodes misclassified")
print(f"The nodes misclassified are : {np.nonzero(misclass[0])}")

## Openflights


We now consider a larger graph. We use the Laplacian eigenmap in dimension 20. We measure proximity between nodes in terms of [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). Equivalently, we work on the embedding in the unit sphere, which can be obtained through the parameter ``normalized`` of the class ``Spectral``.

In [ ]:
graph = openflights

In [ ]:
adjacency = graph.adjacency
position = graph.position
names = graph.names

In [ ]:
image = svg_graph(adjacency, position, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

## To do

* Display the spectrum of the normalized Laplacian.
* Display the same world map with the scores equal to the cosine similarity to **Beijing** in the embedding space.
* List the 10 closest airports to **Madrid** in terms of cosine similarity in the embedding space.
* Display the same world map with the 10 clusters found by k-means in the embedding space.<br> **Hint:** Use the class ``KMeans`` with the appropriate embedding.

In [ ]:
?Spectral

In [ ]:
#spectral = Spectral(20, normalized_laplacian=True)
spectral = Spectral(20, normalized = True, regularization = None, barycenter = False)

In [ ]:
beijing = 1618

In [ ]:
madrid = 572

Display the spectrum of the normalized Laplacian.

In [ ]:
#spectral = Spectral(10, normalized_laplacian=True, barycenter=False, normalized=False, regularization=None)
spectral.fit(adjacency)
eigenvalues_n = spectral.eigenvalues_

In [ ]:
full_eigenvalues = np.hstack(([0],eigenvalues_n))
plt.plot(full_eigenvalues, 'o')
plt.grid()

Display the same world map with the scores equal to the cosine similarity to **Beijing** in the embedding space.

In [ ]:
embedding = spectral.embedding_
score = embedding.dot(embedding[beijing])

In [ ]:
score

In [ ]:
image = svg_graph(adjacency, position, scores = score, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

List the 10 closest airports to Madrid in terms of cosine similarity in the embedding space.

In [ ]:
embedding = spectral.embedding_
score = embedding.dot(embedding[madrid])
image = svg_graph(adjacency, position, scores = score, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

List the 10 closest airports to Madrid in terms of cosine similarity in the embedding space.

In [ ]:
top10 = score.argsort()[-10:][::-1]

In [ ]:
names[top10]

Display the same world map with the 10 clusters found by k-means in the embedding space.

In [ ]:
kmeans = KMeans (10,embedding_method = Spectral(20, normalized=True))
label = kmeans.fit_transform(adjacency)

In [ ]:
image = svg_graph(adjacency, position, labels = label, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

## 2. Digraphs

## Wikipedia Vitals

We now consider a large directed graph. We use the class ``BiSpectral`` (i.e., the directed graph is viewed as a bipartite graph).

In [ ]:
graph = wikivitals

In [ ]:
adjacency = graph.adjacency
names = graph.names
labels = graph.labels_hierarchy
names_labels = graph.names_labels_hierarchy

## To do

* Check that the average cosine similarity between nodes in some set $S$ is given by the square norm of the barycenter of $S$.
* Give the average cosine similarity between nodes in the **Mammals** category.
* Compare with the expected cosine similarity between two nodes sampled uniformly at random.
* Give the average cosine similarity between nodes in some category of your choice.

In [ ]:
spectral = BiSpectral(20, solver=HalkoEig(random_state=42))

In [ ]:
spectral.fit(adjacency)
embedding = spectral.embedding_

In [ ]:
mammal = 274

In [ ]:
names_labels[mammal]

* Check that the average cosine similarity between nodes in some set $S$ is given by the square norm of the barycenter of $S$.

* Give the average cosine similarity between nodes in the Mammals category.

In [ ]:
nodes = np.where(labels == mammal)[0]

In [ ]:
np.linalg.norm(np.mean(embedding[nodes],axis = 0))**2

Average Cosine similarity

In [ ]:
matrix = np.dot( embedding[nodes], np.transpose(embedding[nodes]) )

In [ ]:
np.mean(matrix)

In [ ]:
total = np.sum( matrix[np.triu_indices(92)] )
total = total/4278

In [ ]:
total

Compare with the expected cosine similarity between two nodes sampled uniformly at random.

In [ ]:
two_elem_index = np.random.choice(nodes,2)
print(two_elem_index)
result = embedding[two_elem_index]
print(result[0]@result[1])

In [ ]:
liste = []
for i in range(10000):
    two_elem_index = np.random.choice(np.size(labels),2)
    result = embedding[two_elem_index]
    liste.append(result[0]@result[1])
np.mean(liste)

In [ ]:
result = np.mean (np.dot( embedding, np.transpose(embedding) ) )
result

Give the average cosine similarity between nodes in some category of your choice.

In [ ]:
label = 500
names_labels[label]

In [ ]:
nodes = np.where(labels == label)[0]

In [ ]:
result = np.mean (np.dot( embedding[nodes], np.transpose(embedding[nodes]) ) )
result

On utilise le cosinus parce que celui-ci donne une mesure de proximité
If 1 c'est que il est très proche

## 3. Bigraphs

## Cinema

Finally, we consider the co-embedding of a large bipartite graph. We use the Laplacian eigenmap in dimension 20.

In [ ]:
graph = cinema

In [ ]:
biadjacency = graph.biadjacency
movies = graph.names_row
actors = graph.names_col

## To do

* What is the cosine similarity between **Marion Cotillard** and **The Godfather**  in the embedding space?
* What is the cosine similarity between **The Godfather** and **The Godfather 2** in the embedding space?

In [ ]:
bispectral = BiSpectral(20, regularization=0.5, solver=HalkoEig(random_state=42))

In [ ]:
cotillard = 3814

In [ ]:
actors[cotillard]

In [ ]:
godfather = 69768
godfather2 = 69770

In [ ]:
movies[godfather]

In [ ]:
movies[godfather2]

* What is the cosine similarity between Marion Cotillard and The Godfather in the embedding space?

In [ ]:
bispectral.fit(biadjacency)
embedding_row = bispectral.embedding_row_
embedding_col = bispectral.embedding_col_

In [ ]:
embedding_row[godfather]@embedding_col[cotillard]

In [ ]:
embedding_row[godfather2]@embedding_col[cotillard]